In [136]:
import praw
import pandas as pd

In [137]:
reddit = praw.Reddit(
    client_id='IDPMLFiJcLQO0LSX2_iokw',
    client_secret='YnB6wbk5MdfnVsc4gI5Pc3Aaj1GBLQ',
    user_agent='assignment',
)

In [138]:
reddit.read_only

True

# Collecting post and comments from r/python

In [139]:
posts = reddit.subreddit('python').top(time_filter='week',limit=10)

In [142]:
from praw.models import MoreComments

post_list, comment_list = [],[]

def fetch_comments(comments_obj,post_id):
    
    q = []
    for i,comment in enumerate(comments_obj):
        if i>20:
            break
        if isinstance(comment,MoreComments):
            continue
            
        q.append((comment,0))
    
    while(q):
        comment,lvl = q.pop()
        
        comment_list.append({
            'post_id':post_id,
            'comment_id':comment.id,
            'parent_id':comment.parent_id,
            'body':comment.body,
            'created_on':comment.created_utc,
            'upvotes':comment.score,
            'author_id':comment.author.id if comment.author and hasattr(comment.author,'id') else 'deleted',
            'author_name':comment.author.name if comment.author and hasattr(comment.author,'name') else 'deleted',
        })
        
        if lvl > 0:
            continue
        
        for i,neigh_comment in enumerate(comment.replies):
            if isinstance(neigh_comment,MoreComments):
                continue
            if i >10:
                break
            q.append((neigh_comment,lvl+1))
            
        
            
        
def fetch_posts(subreddit,post_types = ['rising','hot','top week'],limit=10):
    visited = set()
    for post_type in post_types:
        if len(post_type.split(' '))==2 and post_type.split(' ')[0]=='top':
            posts = reddit.subreddit(subreddit).top(time_filter=post_type.split(' ')[1],limit=limit)
        elif post_type =='top':
            posts = reddit.subreddit(subreddit).top(time_filter='week',limit=limit)
        elif post_type=='rising':
            posts = reddit.subreddit(subreddit).rising(limit=limit)
        elif post_type=='hot':
            posts = reddit.subreddit(subreddit).hot(limit=limit)
            
        for post in posts:
            if post.id in visited:
                continue
            visited.add(post.id)
            post_list.append({
                'id':post.id,
                'author':post.author.name if post.author and hasattr(post.author,'name') else 'deleted',
                'author_id':post.author.id if post.author and hasattr(post.author,'id') else 'deleted',
                'total_comments':post.num_comments,
                'upvote':post.score,
                'post_type':post_type,
                'title':post.title,
                'downvote':post.score*(1-post.upvote_ratio),
                'url': post.url,
                'created_on':post.created_utc,
                'subreddit':subreddit,
                'subreddit_id':post.subreddit_id
            })
            
            fetch_comments(post.comments,post.id)

In [143]:
fetch_posts('python',['top'])

In [144]:
post_list

[{'id': '12glkw4',
  'author': 'TheBodyPolitic1',
  'author_id': 'von3w6y2',
  'total_comments': 319,
  'upvote': 588,
  'post_type': 'top',
  'title': "Why didn't Python become popular until long after its creation?",
  'downvote': 35.28000000000003,
  'url': 'https://www.reddit.com/r/Python/comments/12glkw4/why_didnt_python_become_popular_until_long_after/',
  'created_on': 1681051909.0,
  'subreddit': 'python',
  'subreddit_id': 't5_2qh0y'},
 {'id': '12hj9oc',
  'author': 'MetonymyQT',
  'author_id': 'shnqm',
  'total_comments': 73,
  'upvote': 510,
  'post_type': 'top',
  'title': 'Free course: Build a modern API with FastAPI and Python',
  'downvote': 25.50000000000002,
  'url': 'https://www.reddit.com/r/Python/comments/12hj9oc/free_course_build_a_modern_api_with_fastapi_and/',
  'created_on': 1681134311.0,
  'subreddit': 'python',
  'subreddit_id': 't5_2qh0y'},
 {'id': '12egsoz',
  'author': '2broke2code',
  'author_id': 'rs4dqilj',
  'total_comments': 105,
  'upvote': 465,
  'po

In [145]:
comment_list

[{'post_id': '12glkw4',
  'comment_id': 'jfkvwx0',
  'parent_id': 't3_12glkw4',
  'body': "Hardware wasn't ready for Python in that time",
  'created_on': 1681054038.0,
  'upvotes': 5,
  'author_id': '22vat21u',
  'author_name': 'DusikOff'},
 {'post_id': '12glkw4',
  'comment_id': 'jfnha98',
  'parent_id': 't3_12glkw4',
  'body': 'Because Python was developed with the conceit that memory is cheap and plentiful. It took time for consumer level hardware make this true.\n\n[Massive memory overhead: Numbers in Python and how NumPy helps](https://pythonspeed.com/articles/python-integers-memory/)',
  'created_on': 1681095316.0,
  'upvotes': 2,
  'author_id': '4wtjvsh6',
  'author_name': 'FredVIII-DFH'},
 {'post_id': '12glkw4',
  'comment_id': 'jflnf2e',
  'parent_id': 't3_12glkw4',
  'body': 'Perl was *the* scripting language in the early-mid 2000s. I took a look at it and hated the syntax. Python made much more sense to me and a lot of other people but it took time for a critical mass of us

In [146]:
from tinydb import TinyDB

post_db, comment_db = TinyDB('post.json'), TinyDB('comment.json')

In [147]:
post_db.insert_multiple(post_list)
comment_db.insert_multiple(comment_list)

[485,
 486,
 487,
 488,
 489,
 490,
 491,
 492,
 493,
 494,
 495,
 496,
 497,
 498,
 499,
 500,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 523,
 524,
 525,
 526,
 527,
 528,
 529,
 530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 554,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 567,
 568,
 569,
 570,
 571,
 572,
 573,
 574,
 575,
 576,
 577,
 578,
 579,
 580,
 581,
 582,
 583,
 584,
 585,
 586,
 587,
 588,
 589,
 590,
 591,
 592,
 593,
 594,
 595,
 596,
 597,
 598,
 599,
 600,
 601,
 602,
 603,
 604,
 605,
 606,
 607,
 608,
 609,
 610,
 611,
 612,
 613,
 614,
 615,
 616,
 617,
 618,
 619,
 620,
 621,
 622,
 623,
 624,
 625,
 626,
 627,
 628,
 629,
 630,
 631,
 632,
 633,
 634,
 635,
 636,
 637,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 647,
 648,
 649,
 650,
 651

In [148]:
post_db.all()

[{'id': 'g53lxf',
  'author': 'iEslam',
  'author_id': 'g532u',
  'total_comments': 133,
  'upvote': 12335,
  'post_type': 'top',
  'title': "Lad wrote a Python script to download Alexa voice recordings, he didn't expect this email.",
  'downvote': 246.70000000000022,
  'url': 'https://i.redd.it/2s0dj8ob12u41.png',
  'created_on': 1587424299.0,
  'subreddit': 'python',
  'subreddit_id': 't5_2qh0y'},
 {'id': 'hoolsm',
  'author': 'Krukerfluk',
  'author_id': '4t5wv9jj',
  'total_comments': 438,
  'upvote': 9230,
  'post_type': 'top',
  'title': 'This post has:',
  'downvote': 830.6999999999997,
  'url': 'https://www.reddit.com/r/Python/comments/hoolsm/this_post_has/',
  'created_on': 1594386373.0,
  'subreddit': 'python',
  'subreddit_id': 't5_2qh0y'},
 {'id': 'gftejm',
  'author': 'jessjwilliamson',
  'author_id': '2s1icki5',
  'total_comments': 269,
  'upvote': 7870,
  'post_type': 'top',
  'title': 'I redesign the Python logo to make it more modern',
  'downvote': 314.8000000000003,


In [149]:
comment_db.all()

[{'post_id': 'g53lxf',
  'comment_id': 'fo1q2qp',
  'parent_id': 't3_g53lxf',
  'body': 'Think how much spam this guy must have gotten. You exposed your own and his email. lol.',
  'upvotes': 4,
  'author_id': 'deleted',
  'author_name': 'deleted'},
 {'post_id': 'g53lxf',
  'comment_id': 'fo295qq',
  'parent_id': 't3_g53lxf',
  'body': "Nobody's going to mention that Alexa is actually recording what you're saying? .... Seems like an invasion of privacy",
  'upvotes': 6,
  'author_id': '2xpz5uc',
  'author_name': 'GahdDangitBobby'},
 {'post_id': 'g53lxf',
  'comment_id': 'fo3negk',
  'parent_id': 't1_fo295qq',
  'body': "You'd have to be pretty gullible to believe this wasn't happening",
  'upvotes': 2,
  'author_id': '58xh7syf',
  'author_name': 'artFlix'},
 {'post_id': 'g53lxf',
  'comment_id': 'fo39foo',
  'parent_id': 't1_fo295qq',
  'body': "I thought I couldn't be the only one who thinks this is more creepy than adorable. The bigger implications are right under your nose.",
  'upv